In [3]:
from enum import Enum

class Ops(Enum):
    ExpByX = 0,
    Mul = 1,
    Square = 2,
    Conj = 3,
    Frob = 4,
    ExpByXHalf = 5

class X:
    def __init__(self, op, out, first_in, second_in):
        self.op = op
        self.out = out
        self.first_in = first_in
        self.second_in = second_in

p = 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559787
x = 15132376222941642752 # binary form 1101001000000001000000000000000000000000000000010000000000000000 actually its -15132376222941642752
# as we see, using silverman ternary decomposition isnt effectively
# this is x ternary decomposition:
half_of_x = 7566188111470821376 # its x/2  in binary is 110100100000000100000000000000000000000000000001000000000000000
assert(half_of_x*2 == x)
x_decomposition = [
    1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
]
half_of_x_decomposition = [
    1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
]
acc = 1
for bit in x_decomposition[1:]:
    acc *= 2
    acc += bit
assert(acc == x)

acc = 1
for bit in half_of_x_decomposition[1:]:
    acc *= 2
    acc += bit
assert(acc == half_of_x )


r = 52435875175126190479447740508185965837690552500527637822603658699938581184513 

class BLS12HardPartMethod(Enum):
    Simple = 0,
    Naive = 1
    

# the result of Miller loop x is in Fp12* => x^{p^12 - 1} = 1 
# final exponentiation is: x^{(p^12-1)/r} = [x^{(p^6 - 1) * (p^2 + 1)]^{(p^4 - p^2 + 1) / r} = y^{(p^4 - p^2 + 1) / r}
# where y - the result of the easy part of the final exponentiation: y = x^{(p^6 - 1) * (p^2 + 1)} =>
# y^(p^4 - p^2 + 1) = x^{p^12 - 1} = 1 => ord(y) | d = p^4 - p^2 + 1
# assume that we have checked that y != 1, the aim of this script is to prove that in this case Torus arithmetic
# for operation chain for hard part of final exponentiation is exception free (there are no temporary values = 1)
# The reasoning for this is the following: 
# all the partial results occuring the hard part evaluation has the form y^k for some k, and since we assume that 
# y != 1, the only possibility for exception y^k = 1 is ord(y) | k, and ord(y) | d = p^4 - p^2 + 1 and ord(y) > 1
# => ord(y) | gcd(k, d) => gcd(k, d) > 1, and we prove that it is actually never the case
# (except ofcause for the very last step of the chain when k = d/r)

d = p^4 - p^2 + 1
assert(d % r == 0)

def get_chain(hard_part_method):
    if hard_part_method == BLS12HardPartMethod.Simple:
        (f, t0, t1, t2, nil) = (0, 1, 2, 3, 4)
        num_of_vars = 4
        ops_chain = [

           X(Ops.Square, t0, f, nil), X(Ops.ExpByX, t1, f, nil), X(Ops.Conj, t2, f, nil), X(Ops.Mul, t1, t1, t2),
           X(Ops.ExpByX, t2, t1, nil), X(Ops.Conj, t1, t1, nil), X(Ops.Mul, t1, t1, t2), X(Ops.ExpByX, t2, t1, nil),
           X(Ops.Frob, t1, t1, 1), X(Ops.Mul, t1, t1, t2), X(Ops.Mul, f, f, t0), X(Ops.ExpByX, t0, t1, nil),
           X(Ops.ExpByX, t2, t0, nil), X(Ops.Frob, t0, t1, 2), X(Ops.Conj, t1, t1, nil), X(Ops.Mul, t1, t1, t2),
           X(Ops.Mul, t1, t1, t0), X(Ops.Mul, f, f, t1)
        ]
    elif hard_part_method == BLS12HardPartMethod.Naive:
        (f, y0, y1, y2, y3, nil) = (0, 1, 2, 3, 4, 5)
        num_of_vars = 5
        ops_chain = [
            X(Ops.Square, y0, f, nil), X(Ops.ExpByX, y1, y0, nil), X(Ops.ExpByXHalf, y2, y1, nil),
            X(Ops.Conj, y3, f, nil), X(Ops.Mul, y1, y1, y3), X(Ops.Conj, y1, y1, nil), X(Ops.Mul, y1, y1, y2), X(Ops.ExpByX, y2, y1, nil),
            X(Ops.ExpByX, y3, y2, nil), X(Ops.Conj, y1, y1, nil), X(Ops.Mul, y3, y3, y1), X(Ops.Conj, y1, y1, nil),
            X(Ops.Frob, y1, y1, 3), X(Ops.Frob, y2, y2, 2), X(Ops.Mul, y1, y2, y1), X(Ops.ExpByX, y2, y3, nil),
            X(Ops.Mul, y2, y2, y0), X(Ops.Mul, y2, y2, f), X(Ops.Mul, y1, y1, y2), X(Ops.Frob, y2, y3, 1), X(Ops.Mul, f, y1, y2)
        ]
    else:
        raise Exception("The hard part method is not recognized")
    return (ops_chain, num_of_vars)
        
        
def exp_by_x(val):
    res = val
    for bit in x_decomposition[1:]:
        res = res * 2
        if abs(gcd(d, res)) > 1 :
            raise Exception("Torus arithmetic is not exception free 1")
        if bit == 1:
            res = res + val
            if abs(gcd(d, res)) > 1 :
                raise Exception("Torus arithmetic is not exception free 2")
        if bit == -1:
            res = res - val
            if abs(gcd(d, res)) > 1 :
                raise Exception("Torus arithmetic is not exception free 3")
    return -res

def exp_by_half_of_x(val):
    res = val
    for bit in half_of_x_decomposition[1:]:
        res = res * 2
        if abs(gcd(d, res)) > 1 :
            raise Exception("Torus arithmetic is not exception free 1")
        if bit == 1:
            res = res + val
            if abs(gcd(d, res)) > 1 :
                raise Exception("Torus arithmetic is not exception free 2")
        if bit == -1:
            res = res - val
            if abs(gcd(d, res)) > 1 :
                raise Exception("Torus arithmetic is not exception free 3")
    return -res

            
def check(method):
    (ops_chain, num_of_vars) = get_chain(method)
    scratchpad = [0 for i in range(num_of_vars)]
    scratchpad[0] = 1
    num_ops = len(ops_chain)
    
    for (idx, item) in enumerate(ops_chain):
        out = item.out
        if item.op == Ops.ExpByX:
            scratchpad[out] =  exp_by_x(scratchpad[item.first_in])
        elif item.op == Ops.Mul:
            scratchpad[out] = scratchpad[item.first_in] + scratchpad[item.second_in]
        elif item.op == Ops.Square:
            scratchpad[out] = scratchpad[item.first_in] * 2
        elif item.op == Ops.Conj:
            scratchpad[out] = -scratchpad[item.first_in]
        elif item.op == Ops.ExpByXHalf:
            scratchpad[out] = exp_by_half_of_x(scratchpad[item.first_in])
        else:
            scratchpad[out] = scratchpad[item.first_in] * p^item.second_in
        print (scratchpad[0])
        if abs(gcd(d, scratchpad[out])) > 1 and scratchpad[out] != -1 and idx != num_ops - 1:
            raise Exception("Torus arithmetic is not exception free 4")

    actual_result = 3 * d/r
    assert(scratchpad[0] == actual_result)



check(BLS12HardPartMethod.Simple)
check(BLS12HardPartMethod.Naive)
print ("success")


AssertionError: 